In [1]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sqlite3
from apscheduler.schedulers.blocking import BlockingScheduler

def sql_update():
    # show the date today and the date one year ago
    end=datetime.today()
    start= end- relativedelta(years=1)
    print (end)
    print(start)
    # put the stocks what you need to analyze in a dictionary
    stock_dic=dict()
    stocks= ['GOOG','AAPL','AMZN','JPM','MSFT']
    for stock in stocks:
        stock_dic[stock]=pdr.DataReader(stock,"google",start,end)    
    stock_dic[stocks[0]].head()
    # Create a dataframe consisting the close price of the stocks you choose
    stock_df=pd.DataFrame({stock: stock_dic[stock]["Close"]for stock in stocks})
    stock_df.head()
    # Create a dataframe for the log-returns of the stocks in past 1 year
    stock_change=stock_df.apply(lambda x: np.log(x)-np.log(x.shift(1)))
    print(stock_change.head())
    # Write the stock log return dataframe to SQL database
    conn = sqlite3.connect("stockreturn.db")
    stock_change.to_sql("stockreturn", conn, if_exists="replace")



In [ ]:
# schedule the update every minutes.
sched=  BlockingScheduler()
sched.add_job(sql_update,'interval',seconds=60)
sched.start()


2017-08-24 23:10:59.152346
2016-08-24 23:10:59.152346
                AAPL      AMZN      GOOG       JPM      MSFT
Date                                                        
2016-08-24       NaN       NaN       NaN       NaN       NaN
2016-08-25 -0.004267  0.002598 -0.000299  0.001818  0.003789
2016-08-26 -0.005874  0.012799  0.000169  0.002268 -0.002410
2016-08-29 -0.001123  0.002973  0.003386  0.010964  0.001206
2016-08-30 -0.007706 -0.004822 -0.003971  0.008182 -0.003621
